# Definitive Deepfake Detector: A Two-Phase, Unified Training Workflow

This notebook implements the final, most powerful version of our project.

1.  **Workflow**: It uses an efficient **two-phase** process. We first pre-process all raw data (a slow, one-time step), then we train on this prepared data (a fast, repeatable step).
2.  **Architecture**: It uses the full **ResNet -> LSTM -> Transformer** model to analyze videos, and ResNet for images.
3.  **Training**: It trains **one single model** on a unified dataset of **both images and videos together**.
4.  **Output**: It produces a single, versatile model file: `best_unified_model.pth`.

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Before you start, place all your raw media files into a simple, unified folder structure. All real files (images and videos) go in `data/real`, and all fake files go in `data/fake`.

**Required Input Structure:**

In [2]:
%cd /content/drive/MyDrive/deepfake_train
!ls


/content/drive/MyDrive/deepfake_train
best_unified_model.pth	data_processed	  Unified-Deepfake-Training.ipynb
data			requirements.txt


In [4]:
!pip install -r requirements.txt


In [3]:
# This cell installs all necessary packages.
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q
!pip install opencv-python-headless numpy facenet-pytorch tqdm Pillow scikit-learn mediapipe -q

import torch, torch.nn as nn, torchvision.models as models, os, math, cv2, numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from facenet_pytorch import MTCNN # Still needed for comparison/fallback if desired, but not used in new code
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import mediapipe as mp # NEW IMPORT

print("✅ Libraries installed and imported.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
✅ Libraries installed and imported.


## ⚙️ Phase 1: Pre-processing (Run This Cell Only Once)

The following cell will perform the slow face detection and cropping on your entire raw dataset. It will save the clean, ready-to-use files in a new `data_processed` folder.

**This will take a very long time, but you only need to run it once.**

In [ ]:
# --- Configuration ---
SOURCE_DATA_PATH = './data'
PROCESSED_DATA_PATH = './data_processed'
FRAMES_PER_SECOND_TO_EXTRACT = 10 # Your requested frame rate

# --- MediaPipe Initialization ---
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

print(f'Running pre-processing with MediaPipe on device: {device}')

def process_and_save_face(image_pil, dest_path):
    """Uses MediaPipe to detect a face, crop it, resize, and save."""
    image_np = np.array(image_pil)
    # MediaPipe expects BGR, so we convert from PIL's RGB
    results = face_detection.process(cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

    if not results.detections:
        return False # Indicate failure

    # Get bounding box from the first detected face
    detection = results.detections[0]
    box = detection.location_data.relative_bounding_box
    h, w, _ = image_np.shape

    xmin = int(box.xmin * w)
    ymin = int(box.ymin * h)
    width = int(box.width * w)
    height = int(box.height * h)

    # Crop the image using PIL and save
    cropped_face = image_pil.crop((xmin, ymin, xmin + width, ymin + height))
    cropped_face.resize((224, 224)).save(dest_path)
    return True # Indicate success

# This function processes a directory ('real' or 'fake')
def process_directory(class_name):
    source_class_dir = os.path.join(SOURCE_DATA_PATH, class_name)
    processed_class_dir = os.path.join(PROCESSED_DATA_PATH, class_name)
    os.makedirs(processed_class_dir, exist_ok=True)

    if not os.path.isdir(source_class_dir): return

    files_to_process = [f for f in os.listdir(source_class_dir)]

    for file_name in tqdm(files_to_process, desc=f'Processing {class_name} files'):
        source_path = os.path.join(source_class_dir, file_name)

        # --- Handle Videos (with 10 FPS logic) ---
        if any(file_name.lower().endswith(ext) for ext in ['.mp4', '.avi', '.mov']):
            video_id = os.path.splitext(file_name)[0]
            output_folder = os.path.join(processed_class_dir, video_id)
            if os.path.exists(output_folder): continue

            try:
                cap = cv2.VideoCapture(source_path)
                fps = cap.get(cv2.CAP_PROP_FPS)
                if fps == 0: continue

                frame_stride = int(fps / FRAMES_PER_SECOND_TO_EXTRACT)
                if frame_stride == 0: frame_stride = 1

                os.makedirs(output_folder, exist_ok=True)

                current_frame_index = 0
                saved_frame_count = 0
                while True:
                    ret, frame = cap.read()
                    if not ret: break

                    if current_frame_index % frame_stride == 0:
                        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                        save_path = os.path.join(output_folder, f'{saved_frame_count:03d}.jpg')

                        # Use the MediaPipe function to process the frame
                        success = process_and_save_face(frame_pil, save_path)
                        if success:
                            saved_frame_count += 1
                    current_frame_index += 1
                cap.release()
            except Exception as e:
                print(f"Failed to process video {file_name}: {e}")
                continue

        # --- Handle Images (with MediaPipe) ---
        elif any(file_name.lower().endswith(ext) for ext in ['.png', '.jpg', '.jpeg']):
            dest_path = os.path.join(processed_class_dir, file_name)
            if os.path.exists(dest_path): continue
            try:
                img = Image.open(source_path).convert('RGB')
                process_and_save_face(img, dest_path)
            except Exception as e:
                print(f"Failed to process image {file_name}: {e}")
                continue

# Run processing for both 'real' and 'fake' directories
process_directory('real')
process_directory('fake')

print("\n✅ Pre-processing complete!")

Running pre-processing with MediaPipe on device: cuda:0


Processing real files:   0%|          | 0/50 [00:00<?, ?it/s]c:\anaconda\envs\deepfake-env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Processing fake files: 100%|██████████| 50/50 [03:10<00:00,  3.81s/it]


✅ Pre-processing complete!


In [32]:
# --- 1. Final Dataset with Chunking Logic ---
class PreprocessedDataset(Dataset):
    def __init__(self, file_paths, labels, chunk_size=20):
        self.chunk_size = chunk_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        # --- NEW CHUNKING LOGIC ---
        # This new list will hold our final training samples (chunks and images)
        self.samples = []

        print("Scanning and creating chunks from the dataset...")
        for path, label in tqdm(zip(file_paths, labels), total=len(file_paths), desc="Chunking data"):
            if os.path.isdir(path):
                # This is a video, so we create multiple chunks from it
                frame_files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')])
                num_frames = len(frame_files)

                # Create chunks of 'chunk_size' from the video frames
                for i in range(0, num_frames - chunk_size + 1, chunk_size):
                    sample_info = {
                        "type": "video",
                        "path": path,
                        "start_index": i,
                        "label": label
                    }
                    self.samples.append(sample_info)
            else:
                # This is an image, treat it as a single sample
                sample_info = {
                    "type": "image",
                    "path": path,
                    "label": label
                }
                self.samples.append(sample_info)
        print(f"Expanded dataset from {len(file_paths)} files to {len(self.samples)} total training samples (chunks + images).")

    def __len__(self):
        # The length of the dataset is now the total number of chunks and images
        return len(self.samples)

    def __getitem__(self, idx):
        sample_info = self.samples[idx]
        label = sample_info["label"]
        path = sample_info["path"]

        if sample_info["type"] == 'video':
            start_index = sample_info["start_index"]
            frame_files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')])

            # Slice the list to get the correct chunk of frames
            chunk_files = frame_files[start_index : start_index + self.chunk_size]

            try:
                frames = [Image.open(p).convert('RGB') for p in chunk_files]
                processed_data = torch.stack([self.transform(f) for f in frames])
                return processed_data, torch.tensor(label, dtype=torch.float32), 'video'
            except Exception:
                return None
        else: # type is 'image'
            try:
                img = Image.open(path).convert('RGB')
                processed_data = self.transform(img)
                return processed_data, torch.tensor(label, dtype=torch.float32), 'image'
            except Exception:
                return None

# --- Collate Function (No changes needed) ---
def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if not batch: return None
    images, videos, image_labels, video_labels = [], [], [], []
    for data, label, media_type in batch:
        if media_type == 'image': images.append(data); image_labels.append(label)
        else: videos.append(data); video_labels.append(label)
    processed_batch = {}
    if images: processed_batch['image'] = (torch.stack(images), torch.stack(image_labels))
    if videos: processed_batch['video'] = (torch.stack(videos), torch.stack(video_labels))
    return processed_batch

# --- Final Model Definition (No changes needed) ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout, pe = nn.Dropout(p=dropout), torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2], pe[:, 1::2] = torch.sin(position * div_term), torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0).transpose(0, 1))
    def forward(self, x): return self.dropout(x + self.pe[:x.size(0), :])

class ResNetLSTMTransformer(nn.Module):
    def __init__(self, num_classes=1, d_model=512, nhead=8, num_encoder_layers=3, dim_feedforward=1024, freeze_resnet=True):
        super(ResNetLSTMTransformer, self).__init__()
        resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        if freeze_resnet:
            for param in resnet.parameters(): param.requires_grad = False
        self.resnet_features = nn.Sequential(*list(resnet.children())[:-1])
        self.lstm = nn.LSTM(input_size=2048, hidden_size=d_model, num_layers=2, batch_first=True, bidirectional=True)
        self.lstm_output_layer = nn.Linear(d_model * 2, d_model)
        self.pos_encoder = PositionalEncoding(d_model=d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_encoder_layers)
        self.classifier_video = nn.Sequential(nn.Linear(d_model, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, num_classes))
        self.classifier_image = nn.Sequential(nn.Linear(2048, 256), nn.ReLU(), nn.Dropout(0.5), nn.Linear(256, num_classes))
        self.d_model = d_model

    def forward(self, x):
        is_video = x.dim() == 5
        if is_video:
            batch_size, seq_length, c, h, w = x.shape
            x = x.view(batch_size * seq_length, c, h, w)
        features = self.resnet_features(x).view(x.size(0), -1)
        if is_video:
            features = features.view(batch_size, seq_length, -1)
            features = self.lstm_output_layer(self.lstm(features)[0])
            features = self.pos_encoder(features)
            output = self.classifier_video(self.transformer_encoder(features).mean(dim=1))
        else:
            output = self.classifier_image(features)
        return output

print("✅ Training components defined.")

✅ Training components defined.


In [36]:
# --- ⚙️ TRAINING CONFIGURATION ---
DATA_PATH = '/content/drive/MyDrive/deepfake_train/data_processed'
EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 1e-7
NUM_FRAMES = 20
MODEL_SAVE_PATH = '/content/drive/MyDrive/deepfake_train/best_unified_model.pth'

# --- Setup Device ---
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- File Collection (from processed data) ---
real_path, fake_path = os.path.join(DATA_PATH, 'real'), os.path.join(DATA_PATH, 'fake')
if not os.path.exists(real_path) or not os.path.exists(fake_path):
    print(f"❌ CRITICAL ERROR: Pre-processed data not found. Please run the Phase 1 cell first.")
else:
    real_files, fake_files = [os.path.join(real_path, f) for f in os.listdir(real_path)], [os.path.join(fake_path, f) for f in os.listdir(fake_path)]
    files, labels = real_files + fake_files, [0] * len(real_files) + [1] * len(fake_files)

    if not files:
        print("❌ CRITICAL ERROR: No pre-processed files found. Please run the Phase 1 cell.")
    else:
        print(f"Found {len(files)} total pre-processed items to train on together.")
        train_files, val_files, train_labels, val_labels = train_test_split(files, labels, test_size=0.2, stratify=labels, random_state=42)

        train_dataset, val_dataset = PreprocessedDataset(train_files, train_labels, NUM_FRAMES), PreprocessedDataset(val_files, val_labels, NUM_FRAMES)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=custom_collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=custom_collate_fn)
        print("✅ Datasets and DataLoaders created successfully.")

Using device: cuda:0
Found 100 total pre-processed items to train on together.
Scanning and creating chunks from the dataset...


Chunking data: 100%|██████████| 80/80 [00:00<00:00, 346.17it/s]


Expanded dataset from 80 files to 615 total training samples (chunks + images).
Scanning and creating chunks from the dataset...


Chunking data: 100%|██████████| 20/20 [00:00<00:00, 301.59it/s]

Expanded dataset from 20 files to 188 total training samples (chunks + images).
✅ Datasets and DataLoaders created successfully.


In [37]:
# === RESUME FROM CHECKPOINT (if available) ===
import os
import torch

# device variable is set in Cell 6 already; ensure it's available
try:
    device
except NameError:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Device for training:", device)
print("Looking for checkpoint at:", MODEL_SAVE_PATH)

start_epoch = 0
checkpoint = None
if os.path.exists(MODEL_SAVE_PATH):
    print("Found checkpoint. Loading...")
    checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
    # checkpoint might be a state_dict or a dict with 'state_dict' and other keys
else:
    print("No checkpoint found; training from scratch.")

# Instantiate model architecture (same code as later in the notebook)
model = ResNetLSTMTransformer().to(device)

# Load weights if checkpoint contains state_dict
if checkpoint is not None:
    try:
        # Most common case: saved state_dict directly
        model.load_state_dict(checkpoint)
        print("✅ Loaded checkpoint as state_dict.")
    except Exception as e1:
        try:
            # Another common pattern: checkpoint is a dict with 'state_dict'
            model.load_state_dict(checkpoint.get('state_dict', checkpoint))
            print("✅ Loaded checkpoint['state_dict'].")
        except Exception as e2:
            print("⚠️ Could not load checkpoint automatically:", e2)
            raise

# Prepare loss and optimizer as in the original training cell
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# If checkpoint also stores optimizer and epoch, restore them
if checkpoint is not None and isinstance(checkpoint, dict):
    if 'optimizer' in checkpoint:
        try:
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("✅ Loaded optimizer state from checkpoint.")
        except Exception as e:
            print("⚠️ Could not load optimizer state:", e)
    if 'epoch' in checkpoint:
        start_epoch = checkpoint['epoch'] + 1
        print(f"Will resume from epoch {start_epoch}")

print("Model and optimizer ready. start_epoch =", start_epoch)


Device for training: cuda:0
Looking for checkpoint at: /content/drive/MyDrive/deepfake_train/best_unified_model.pth
Found checkpoint. Loading...
✅ Loaded checkpoint as state_dict.
Model and optimizer ready. start_epoch = 0


In [38]:
# --- 🧠 TRAINING LOOP ---

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss, total_acc, total_samples = 0.0, 0, 0
    for batch in tqdm(dataloader, desc="Training"):
        if batch is None: continue
        for media_type, (inputs, labels) in batch.items():
            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * inputs.size(0)
            total_acc += ((torch.sigmoid(outputs) > 0.5) == labels).sum().item()
            total_samples += labels.size(0)
    return total_loss/total_samples if total_samples > 0 else 0, total_acc/total_samples if total_samples > 0 else 0

def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()
    total_loss, total_acc, total_samples = 0.0, 0, 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validating"):
            if batch is None: continue
            for media_type, (inputs, labels) in batch.items():
                inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item() * inputs.size(0)
                total_acc += ((torch.sigmoid(outputs) > 0.5) == labels).sum().item()
                total_samples += labels.size(0)
    return total_loss/total_samples if total_samples > 0 else 0, total_acc/total_samples if total_samples > 0 else 0

model = ResNetLSTMTransformer().to(device)
criterion, optimizer = nn.BCEWithLogitsLoss(), torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{EPOCHS} -> Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
    if val_loss > 0 and val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"✅ Saved new best model to {MODEL_SAVE_PATH}")

Validating: 100%|██████████| 12/12 [00:33<00:00,  2.82s/it]


Epoch 1/10 -> Train Loss: 0.6954, Acc: 0.5350 | Val Loss: 0.6833, Acc: 0.5798
✅ Saved new best model to /content/drive/MyDrive/deepfake_train/best_unified_model.pth


Validating: 100%|██████████| 12/12 [00:34<00:00,  2.83s/it]


Epoch 2/10 -> Train Loss: 0.6949, Acc: 0.5220 | Val Loss: 0.6834, Acc: 0.5798


Validating: 100%|██████████| 12/12 [00:33<00:00,  2.77s/it]


...

Epoch 8/10 -> Train Loss: 0.7042, Acc: 0.5187 | Val Loss: 0.6838, Acc: 0.5798


Validating: 100%|██████████| 12/12 [00:33<00:00,  2.83s/it]


Epoch 9/10 -> Train Loss: 0.7036, Acc: 0.5268 | Val Loss: 0.6838, Acc: 0.5798


Validating: 100%|██████████| 12/12 [00:33<00:00,  2.77s/it]

Epoch 10/10 -> Train Loss: 0.6921, Acc: 0.5317 | Val Loss: 0.6838, Acc: 0.5798


In [18]:
# Restarting kernel recommended after many tries. Then run:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi


Sun Oct 12 14:39:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             34W /   70W |    8708MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# --- 🚀 PREDICTION ---
import os # Import the os module

def predict_raw_file(file_path, model_path):
    pred_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {pred_device}")

    mtcnn = MTCNN(image_size=224, margin=20, device=pred_device, keep_all=False, post_process=False)
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    pred_model = ResNetLSTMTransformer().to(pred_device)
    pred_model.load_state_dict(torch.load(model_path, map_location=pred_device))
    pred_model.eval()

    tensor_data, media_type = None, ''
    is_video = any(file_path.lower().endswith(ext) for ext in ['.mp4', '.avi', '.mov'])

    if is_video:
        media_type = 'video'
        cap = cv2.VideoCapture(file_path)
        frames = []
        if cap.isOpened():
            total_f = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            indices = np.linspace(0, total_f - 1, 20, dtype=int)
            for i in sorted(indices):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                ret, frame = cap.read()
                if ret: frames.append(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
            cap.release()
        if frames:
            faces = mtcnn(frames)
            if faces is not None and len(faces) > 0:
                face_frames = [transforms.ToPILImage()((f+1)/2) for f in faces if f is not None]
                if len(face_frames) < 20: face_frames.extend([face_frames[-1]]*(20-len(face_frames)))
                tensor_data = torch.stack([transform(f) for f in face_frames])
    else:
        media_type = 'image'
        try:
            face = mtcnn(Image.open(file_path).convert('RGB'))
            if face is not None: tensor_data = transform((face + 1) / 2)
        except Exception: tensor_data = None

    if tensor_data is None or (is_video and tensor_data.size(0) == 0):
        print("❌ Error: Could not detect a face in the provided file.")
        return

    with torch.no_grad():
        if is_video:
             # Add a batch dimension for video input
            output = pred_model(tensor_data.unsqueeze(0).to(pred_device))
        else:
            output = pred_model(tensor_data.unsqueeze(0).to(pred_device))
        prob_fake = torch.sigmoid(output).item()

    UNCERTAIN_LOW, UNCERTAIN_HIGH = 0.40, 0.60
    if UNCERTAIN_LOW < prob_fake < UNCERTAIN_HIGH:
        prediction, confidence_str = "Unable to Predict", ""
    elif prob_fake >= UNCERTAIN_HIGH:
        prediction, confidence_str = "FAKE", f"({prob_fake * 100:.2f}% confidence)"
    else:
        prediction, confidence_str = "REAL", f"({(1 - prob_fake) * 100:.2f}% confidence)"

    print(f"\n--- Prediction Result for {media_type} ---")
    print(f"File: {os.path.basename(file_path)}")
    print(f"Result: {prediction} {confidence_str}")

# --- ⚠️ USAGE ---
test_model_path = '/content/drive/MyDrive/deepfake_train/best_unified_model.pth' # Assumes model is saved here
test_file_path = '/content/drive/MyDrive/deepfake_train/we.jpg' # <-- CHANGE this to the path of your test file (image or video)

if os.path.exists(test_model_path) and os.path.exists(test_file_path):
    predict_raw_file(test_file_path, test_model_path)
else:
    print(f"❌ Error: Check that your model path ('{test_model_path}') and file path ('{test_file_path}') are correct.")

❌ Error: Check that your model path ('/content/drive/MyDrive/deepfake_train/best_unified_model.pth') and file path ('/content/drive/MyDrive/deepfake_train/we.jpg') are correct.
